# Region-scale glacier analysis


This notebook will walk you through steps to read in and organize velocity data in a raster format using xarray and rioxarray tools

First, lets install the python libraries that were listed on the [Software](software.ipynb) page:

# new notebook structure (still working on)**

- brief description of notebook at top (section 1)
- raster datastream 1, ITS_LIVE geotiff ingesting and organizing (section 2)
    - look at dims, vars, crs etc
- raster datastream 2, ITS_IVE netcdf ingesting and organizing (section 3)
    - look at dims, vars, crs etc
- read in vector data (section 4)
- working with projections (section 5)
    - raster datasets have different projections (UTM and Asia_North_Lambert_Conformal_Conic)
    - should i put them into same CRS or have a different projected geodataframe of glaciers for each velocity ds, and totally separate workflows...?
        - if same crs, which prefereable btw UTM and asia norht lambert conformal conic
- once both raster types are in workable formats, start on reading in vectors etc (section 6...)


In [ ]:
import geopandas as gpd
import os
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube

In [ ]:
!pwd

In [ ]:
gen_path = '/Users/emarshall/Desktop/siparcs/xr_book1/'

## Raster datastream 1: ITS_LIVE (Geotiff)

This section contains a workflow for reading in and organizing ITS_LIVE glacier velocity data that is accessed in geotiff format from S3

The velocity data we are using is broken into individual velocity components. That means that for the same spatial footprint, we have a file containing ice movement in the x direction and a file containing ice movement in the y direction. We need information from both of these files so we write a function to bring both files into the jupyter notebook, then organize them so that we can see the movement of ice in both the x and y directions as well as the magnitude of the ice velocity (speed). 



In [ ]:
def components_to_speed(vx_path, vy_path):
    '''this function reads in x,y components of velocity, generates speed variable. return xarray
    dataset w/ x,y, speed variables. function will break if vx,vy objects don"t have same x,y coords'''
    
    vy_da = rxr.open_rasterio(vy_path, masked=False).squeeze()
    vx_da = rxr.open_rasterio(vx_path, masked=False).squeeze()
    
    ds_gen = xr.Dataset()
    ds_gen['vx'] = vx_da
    ds_gen['vy'] = vy_da
    sp = np.sqrt((ds_gen['vx'].data**2) + ds_gen['vy'].data**2)
    ds_gen['sp'] = (['x','y'], sp.T)
    
    return ds_gen

Let's break down what exactly the above function is doing:

First, we see that it takes two inputs: vx_path and vy_path. These paths point to where on our computer the different files are stored. 

In the first two lines of the function we use rioxarray to read in the x- and the y-component files as **xarray.DataArrays**

After that, we initialize a new object, *ds_gen*, which is a **xarray.DataSet**. We then add a variable to ds_gen called 'vx' and assign the vx_da object to that variable. We do the same for vx_da. Now, we have made a dataset that is composed of the two data arrays that we read in from file. 

We are also interested in speed, so we take the equation for computing magnitude of velocity and add a third variable (DataArray) to our Dataset. 

This will add a variable defined by the equation:

            vv = (vx^2 + vy^2)^1/2

Let's execute the function and take a look at the object it returns

First, define the inputs to your function. These are the paths to the x and the y data on your computer:

In [ ]:
n45_vy_path = gen_path + 'mynewbook/gardner_data/N45_0240m_vy.tiff'
n45_vx_path = gen_path + 'mynewbook/gardner_data/N45_0240m_vx.tiff'


And run the function: 

In [ ]:
ds_45n = components_to_speed(n45_vx_path, n45_vy_path)


In [ ]:
ds_45n

In [ ]:
type(ds_45n)


What is the CRS of this object?

In [ ]:
ds_45n.spatial_ref.attrs

In [ ]:
print(ds_45n.dims)
print('---')
print(ds_45n.coords)
print('---')
print(ds_45n.variables)
print('---')
print(ds_45n.attrs)

## Raster datastream 2: ITS_LIVE (netcdf)

This section contains a workflow for reading in and organizing ITS_LIVE glacier velocity data that is accessed in netcdf format from the NSIDC DAAC. Whereas before, we needed to build a the magnitude of velocity variable from the velocity component variables (individual geotiff files), the netcdf file contains a variable for magnitude of velocity as well as many other variables representing land cover types, error estimates and metadata

In [ ]:
itslive = rxr.open_rasterio(gen_path[:-9] + '/data/HMA_G0120_0000.nc').squeeze()

In [ ]:
itslive

What is the CRS of this object?

There are two ways we can check that. First, by using the `rio.crs` accessor:

In [ ]:
itslive.rio.crs

The netcdf object is in a different CRS than the geotiff object. Because **Asia North Lambert Conformal Conic** covers a larger spatial extent than a single UTM zone (the projection of the geotiff object), we will use that projection.
*add link to good explainer page?*

In [ ]:
itslive.dims

In [ ]:
itslive.coords

## Vector data 

In [ ]:
#read in vector data 
se_asia = gpd.read_file(gen_path[:-9] + 'data/nsidc0770_15.rgi60.SouthAsiaEast/15_rgi60_SouthAsiaEast.shp')

How many glaciers are in this dataframe?

In [ ]:
len(se_asia['RGIId'])

What coordinate reference system is this dataframe in? 

In [ ]:
se_asia.crs

The vector dataset is in WGS 84, meaning that its coordinates are in degrees latitude and longitude rather than meters N and E. We will project this dataset to match the projection of the netcdf dataset.

## Handling projections

Let's project this dataframe to match the CRS of the itslive dataset

In [ ]:
#not sure why but this didn't work for me specifying epsg code, had to specify full description
se_asia_prj = se_asia.to_crs('+proj=lcc +lat_1=15 +lat_2=65 +lat_0=30 +lon_0=95 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs')
se_asia_prj

In [ ]:
se_asia_prj['RGI_int'] = se_asia_prj['RGIId'].str.slice(9,).replace('.','_')
se_asia_prj['RGI_int'] = se_asia_prj.RGI_int.apply(lambda x: '15' + x)
se_asia_prj

In [ ]:
se_asia_prj.loc[se_asia_prj['Area'] > 5.]['RGI_int']

In [ ]:
def rasterize_vector(gpdf_prj, raster_obj): 
    
    #read in gpdf from shp file
    #gpdf = gpd.read_file(vector_path)
    #project to local utm
    #gpdf_utm = gpdf.to_crs(f'EPSG:{utm_code}')
    #use index as a unique key for each glacier
    gpdf_sub = gpdf_prj.loc[gpdf_prj['Area'] > 5.]
    gpdf_sub['RGI_int'] = gpdf_sub.index.astype(int) # RGI_int corresponds to RGI ID but in int form ie 15.00001 = 1500001
    #print(gpdf_utm['Integer_ID'])
    
    #rasterize glacier vector by unique id 

    out_grid = make_geocube(
            vector_data = gpdf_sub,
            measurements = ['RGI_int'],
            like = raster_obj['v'] #need to specify a var here, not sure best way to do that
            )
    
    #now merge the rasterized vector and the original raster togehter into a geocube
    out_grid['speed'] = (raster_obj.dims, raster_obj.v.values, raster_obj.attrs, raster_obj.encoding)
    
    #now, get velocity statistics of each 'region' (integer) using the mask
    #grouped_ID = out_grid.drop('spatial_ref').groupby(out_grid.Integer_ID)

    #compute zonal stats groupedd by ID
    #grid_mean_sp = grouped_ID.mean().rename({'speed': 'speed_mean'})
    #grid_min_sp = grouped_ID.min().rename({'speed': 'speed_min'})
    #grid_max_sp = grouped_ID.max().rename({'speed': 'speed_max'})
    #grid_std_sp = grouped_ID.max().rename({'speed': 'speed_std'})
    
    #merge each zonal stat xr obj into a single xr obj, convert to pandas df
    #zonal_stats = xr.merge([grid_mean_sp, grid_min_sp, grid_max_sp, grid_std_sp]).to_dataframe()
    #zonal_stats = zonal_stats.reset_index()
    
   # return zonal_stats
    return out_grid

In [ ]:
rasterize_vector_seasia = rasterize_vector(se_asia_prj, itslive)
rasterize_vector_seasia

In [ ]:
np.count_nonzero(~np.isnan(rasterize_vector_seasia.RGI_int))

In [ ]:
#now, get velocity statistics of each 'region' (integer) using the mask
grouped_ID = rasterize_vector_seasia.drop('spatial_ref').groupby(rasterize_vector_seasia.RGI_int)


In [ ]:
#compute zonal stats groupedd by ID
grid_mean_sp = grouped_ID.mean().rename({'speed': 'speed_mean'})
grid_min_sp = grouped_ID.min().rename({'speed': 'speed_min'})
grid_max_sp = grouped_ID.max().rename({'speed': 'speed_max'})
grid_std_sp = grouped_ID.max().rename({'speed': 'speed_std'})
   

In [ ]:
#merge each zonal stat xr obj into a single xr obj, convert to pandas df
zonal_stats = xr.merge([grid_mean_sp, grid_min_sp, grid_max_sp, grid_std_sp]).to_dataframe()
zonal_stats = zonal_stats.reset_index()
    
zonal_stats

In [ ]:
len(zonal_stats['RGI_int'])

In [ ]:
#project to utm
#se_asia_utm = se_asia.to_crs('EPSG:32645')
#make a col in df that is a unique integer ID (from index) for each glacier
#se_asia_utm['Integer_ID'] = se_asia_utm.index.astype(int)
#double checking that all glaciers are assigned an ID
#se_asia_utm.plot.scatter(x='Integer_ID', y='Area')

The plot above shows the mean ice speed of every glacier in the geodataframe object, **se_asia**, that lies within the spatial extent the velocity object.

In [ ]:
#rasterize glacier vector by unique id 
#
out_grid_se_asia = make_geocube(
            vector_data = se_asia_prj.loc[se_asia_prj['Area'] > 5.],
            measurements = ['RGI_int'],
            like = itslive['v']
)

In [ ]:
#now merge the rasterized vector and the original raster togehter into a geocube
out_grid_se_asia['speed'] = (itslive.dims, itslive.v.values, itslive.attrs, itslive.encoding)
out_grid_se_asia

In [ ]:
#trying to figure out why 1300 glaciers or so get dropped
print(len(out_grid_se_asia.RGI_int))

In [ ]:
#now, get velocity statistics of each 'region' (integer) using the mask
grouped_ID = out_grid_se_asia.drop('spatial_ref').groupby(out_grid_se_asia.RGI_int)
grouped_ID

In [ ]:
grid_mean_sp = grouped_ID.mean().rename({'speed': 'speed_mean'})
grid_median_sp = grouped_ID.median().rename({'speed': 'speed_median'})
grid_min_sp = grouped_ID.min().rename({'speed': 'speed_min'})
grid_max_sp = grouped_ID.max().rename({'speed': 'speed_max'})
grid_std_sp = grouped_ID.max().rename({'speed': 'speed_std'})

In [ ]:
zonal_stats = xr.merge([grid_mean_sp, grid_median_sp, grid_min_sp, grid_max_sp, grid_std_sp]).to_dataframe()
zonal_stats = zonal_stats.reset_index()
zonal_stats

In [ ]:
#now, trying to merge zonal stats df back with original glacier df on integer_ID col
se_asia_glacier_data = se_asia_prj.loc[se_asia_prj['Area'] > 5.].merge(zonal_stats, on='RGI_int')


In [ ]:
zonal_stats['speed_mean']

fig, ax = plt.subplots()
se_asia_glacier_data.plot.scatter(x='RGI_int',y = 'speed_mean', c = 'darkblue', ax=ax)


In [ ]:
zonal_stats['speed_mean'].min()

In [ ]:
se_asia_glacier_data.plot(column='speed_mean', legend=True)